.md explanations and justifications wip

In [1]:
from utils.sim_utils import *
import pandas as pd
from typing import Optional

def load_data(movies_path, ratings_path):
    movies = pd.read_csv(movies_path)
    ratings = pd.read_csv(ratings_path)
    return movies, ratings


def create_rating_matrix(ratings: pd.DataFrame) -> pd.DataFrame:
    return ratings.pivot(index='userId', columns='movieId', values='rating')

SIMILARITY_FUNCS = {
    'pearson': pearson,
    'constrained_pearson': constrained_pearson,
    'cosine': cosine,
    'jaccard': jaccard,
    'euclidean': euclidean,
    'manhattan': manhattan,
    'fast_cosine': vectorized_cosine,
    'fast_pearson': vectorized_pearson,
}

In [2]:
def compute_similarity_matrix(rating_matrix, method, shrinkage: Optional[float] = None):
    if method not in SIMILARITY_FUNCS:
        raise ValueError(f"Invalid method: {method}")
    func = SIMILARITY_FUNCS.get(method)
    if func is None:
        raise ValueError(f"Invalid method: {method}")
    if shrinkage is not None and method != 'constrained_pearson':
        raise ValueError(f"Shrinkage is only supported for 'constrained_pearson' method, not {method}")

    # For large matrices, use a sample for faster computation
    sample_size = 200  # Adjust this value based on your needs

    entities = rating_matrix.index
    n = len(entities)

    # If matrix is too large, use a sample
    if n > sample_size and method != 'cosine':
        print(f"Using a sample of {sample_size} entities for similarity computation")
        # Sample entities
        sampled_indices = np.random.choice(n, size=sample_size, replace=False)
        sampled_entities = entities[sampled_indices]
        rating_matrix_sample = rating_matrix.loc[sampled_entities]
        entities = sampled_entities
        n = len(entities)
        data = rating_matrix_sample.values
    else:
        data = rating_matrix.values

    # For cosine similarity, use sklearn's optimized implementation
    if method == 'cosine':
        from sklearn.metrics.pairwise import cosine_similarity
        # Replace NaN with 0 for cosine similarity
        data_filled = np.nan_to_num(data)
        sim_mat = cosine_similarity(data_filled)
        return pd.DataFrame(sim_mat, index=entities, columns=entities)

    # For other methods, use our implementation
    sim_mat = np.zeros((n, n))

    # Compute similarities
    for i in range(n):
        # Set diagonal to 1 (self-similarity)
        sim_mat[i, i] = 1.0

        # Only compute upper triangle (symmetric matrix)
        for j in range(i+1, n):
            u = data[i, :]
            v = data[j, :]

            if method == 'constrained_pearson' and shrinkage is not None:
                score = func(u, v, shrinkage)
            else:
                score = func(u, v)

            # Set both entries (symmetric matrix)
            sim_mat[i, j] = score
            sim_mat[j, i] = score

    return pd.DataFrame(sim_mat, index=entities, columns=entities)

def get_top_k_neighbors(sim_matrix, entity_id, k):
    if entity_id not in sim_matrix.index:
        return pd.Series(dtype=float)
    if entity_id in sim_matrix.columns:
        scores = sim_matrix.loc[entity_id].drop(index=entity_id, errors='ignore')
        return scores.nlargest(k)
    else:
        return pd.Series(dtype=float)

In [3]:
def predict_rating_user(user_id, item_id, rating_matrix, sim_matrix, k=10):
    if user_id not in rating_matrix.index:
        return rating_matrix.values.mean()  # Return global mean if user not found

    # Check if item exists in the rating matrix
    if item_id not in rating_matrix.columns:
        return rating_matrix.loc[user_id].mean()  # Return user's mean if item not found

    # Check if user exists in similarity matrix
    if user_id not in sim_matrix.index:
        return rating_matrix.loc[user_id].mean()  # Return user's mean if user not in similarity matrix

    neighbors = get_top_k_neighbors(sim_matrix, user_id, k)
    if len(neighbors) == 0:
        return rating_matrix.loc[user_id].mean()

    # Only consider neighbors who rated the item
    neigh_ratings = rating_matrix.loc[neighbors.index, item_id]

    # Create a mask for non-NaN values in neighbor ratings
    mask = ~neigh_ratings.isna()
    sims = neighbors[mask.values]  # Use .values to convert Series mask to array
    ratings = neigh_ratings[mask]

    # If no valid neighbors, return user's mean rating
    if len(sims) == 0 or sims.abs().sum() == 0:
        return rating_matrix.loc[user_id].mean()

    neigh_means = rating_matrix.loc[sims.index].mean(axis=1)
    numer = ((ratings - neigh_means) * sims).sum()
    denom = sims.abs().sum()
    return rating_matrix.loc[user_id].mean() + numer / denom if denom else rating_matrix.loc[user_id].mean()

In [4]:
def predict_rating_item(user_id, item_id, rating_matrix, item_sim, k = 20):
    """Item-based prediction with fallback to user mean."""
    if user_id not in rating_matrix.index:
        return rating_matrix.values.mean()  # Return global mean if user not found

    user_ratings = rating_matrix.loc[user_id].dropna()
    if len(user_ratings) == 0:
        return rating_matrix.values.mean()  # Return global mean if user hasn't rated anything

    if item_id not in item_sim.index:
        return user_ratings.mean()  # Return user's mean if item not in similarity matrix

    # Make sure we're only getting similarities for items the user has rated
    rated_items = list(set(user_ratings.index).intersection(set(item_sim.columns)))
    if not rated_items:
        return user_ratings.mean()  # Return user's mean if no overlap between rated items and similarity matrix

    try:
        sims = item_sim.loc[item_id, rated_items]
        topk = sims.nlargest(min(k, len(sims)))
    except KeyError:
        # Handle case where item_id is not in similarity matrix
        return user_ratings.mean()

    if len(topk) == 0 or topk.abs().sum() == 0:
        return user_ratings.mean()

    try:
        numer = (user_ratings[topk.index] * topk).sum()
        denom = topk.abs().sum()
        return numer/denom
    except KeyError:
        # Handle case where some indices in topk are not in user_ratings
        return user_ratings.mean()

In [5]:
def matrix_factorization(R, k=20, alpha=0.005, beta=0.02, iterations=50):
    """
    Train matrix factorization via SGD:
    Minimize sum_{(u,i) in R} (r_ui - P_u^T Q_i)^2 + beta*(||P_u||^2 + ||Q_i||^2)
    """
    # Initialize
    num_users, num_items = R.shape
    P = np.random.normal(scale=1./k, size=(num_users, k))
    Q = np.random.normal(scale=1./k, size=(num_items, k))

    # Extract known ratings
    users, items = np.where(~np.isnan(R.values))
    ratings = R.values[users, items]

    # Use fewer iterations for faster execution
    print(f"Matrix factorization with {iterations} iterations...")
    for iter in range(iterations):
        # Shuffle the data
        indices = np.arange(len(ratings))
        np.random.shuffle(indices)

        # Process in mini-batches for speed
        batch_size = 1000
        num_batches = len(indices) // batch_size + 1

        total_error = 0
        for batch in range(num_batches):
            batch_indices = indices[batch*batch_size:min((batch+1)*batch_size, len(indices))]

            if len(batch_indices) == 0:
                continue

            batch_users = users[batch_indices]
            batch_items = items[batch_indices]
            batch_ratings = ratings[batch_indices]

            for idx in range(len(batch_indices)):
                u, i, r = batch_users[idx], batch_items[idx], batch_ratings[idx]
                pred = P[u, :].dot(Q[i, :].T)
                e = r - pred
                total_error += e**2

                # Update factors
                P[u, :] += alpha * (e * Q[i, :] - beta * P[u, :])
                Q[i, :] += alpha * (e * P[u, :] - beta * Q[i, :])

        # Print progress every few iterations
        if (iter + 1) % 5 == 0 or iter == 0:
            rmse = np.sqrt(total_error / len(ratings))
            print(f"  Iteration {iter+1}/{iterations}, RMSE: {rmse:.4f}")

    return P, Q

def predict_rating_mf(user_id, item_id, R, P, Q):
    """Predict rating using matrix factorization."""
    # Check if user and item exist in the matrices
    if user_id not in R.index or item_id not in R.columns:
        # Return global mean if user or item not found
        return R.stack().mean()
    try:
        # Get indices
        user_idx = list(R.index).index(user_id)
        item_idx = list(R.columns).index(item_id)
        pred = P[user_idx, :].dot(Q[item_idx, :].T)
        # Clip to rating range [0.5, 5]
        return np.clip(pred, 0.5, 5.0)
    except (ValueError, IndexError) as e:
        print(f"Error predicting for user {user_id}, item {item_id}: {e}")
        return R.stack().mean()

In [7]:
def rmse(preds, targets):
    """Compute Root Mean Squared Error ensuring arrays have the same shape."""
    # Check if arrays have the same length
    if len(preds) != len(targets):
        min_len = min(len(preds), len(targets))
        preds = preds[:min_len]
        targets = targets[:min_len]
        print(f"Warning: Arrays had different lengths. Using only the first {min_len} elements.")

    # Check if arrays are empty
    if len(preds) == 0 or len(targets) == 0:
        return float('nan')

    return np.sqrt(np.mean((preds - targets) ** 2))

def main():
    import os
    import time
    from tqdm import tqdm  # for progress bars

    movies_path = '../data/movies.csv'
    ratings_path = '../data/ratings.csv'

    start_time = time.time()
    movies, ratings = load_data(movies_path, ratings_path)
    print(f"Data loaded in {time.time() - start_time:.2f} seconds")

    # Take a smaller sample for faster execution
    print("Taking a smaller sample for faster execution...")
    ratings = ratings.sample(frac=0.3, random_state=42)

    # Split into train/test (leave-one-out per user)
    print("Splitting data into train/test sets...")
    start_time = time.time()
    test_idx = []
    for uid, group in ratings.groupby('userId'):
        if len(group) < 2:
            continue
        # randomly select one rating for test
        sample = group.sample(n=1, random_state=42)
        test_idx.extend(sample.index)
    test_ratings = ratings.loc[test_idx]
    train_ratings = ratings.drop(index=test_idx)
    print(f"Data split in {time.time() - start_time:.2f} seconds")
    print(f"Train set: {len(train_ratings)} ratings, Test set: {len(test_ratings)} ratings")

    # Create rating matrix
    print("Creating rating matrix...")
    start_time = time.time()
    train_matrix = create_rating_matrix(train_ratings)
    print(f"Rating matrix created in {time.time() - start_time:.2f} seconds")
    print(f"Matrix shape: {train_matrix.shape}")

    # User-based CF
    print("\nComputing user similarity matrix...")
    start_time = time.time()
    user_sim = compute_similarity_matrix(train_matrix, method='constrained_pearson', shrinkage=10)
    print(f"User similarity matrix computed in {time.time() - start_time:.2f} seconds")

    print("Evaluating user-based CF...")
    start_time = time.time()
    y_true, y_pred_u = [], []

    # Use a smaller sample for testing if the test set is large
    eval_test = test_ratings
    if len(test_ratings) > 1000:
        eval_test = test_ratings.sample(n=1000, random_state=42)
        print(f"Using {len(eval_test)} samples for evaluation")

    try:
        for _, row in tqdm(eval_test.iterrows(), total=len(eval_test)):
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_user(u, i, train_matrix, user_sim, k=20)
                if not np.isnan(pred):
                    y_true.append(row['rating'])
                    y_pred_u.append(pred)
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue
    except NameError:
        for _, row in eval_test.iterrows():
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_user(u, i, train_matrix, user_sim, k=20)
                if not np.isnan(pred):
                    y_true.append(row['rating'])
                    y_pred_u.append(pred)
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue

    print(f"User-based CF evaluation completed in {time.time() - start_time:.2f} seconds")
    print(f"User-based CF RMSE: {rmse(np.array(y_pred_u), np.array(y_true)):.4f}")

    # Item-based CF
    print("\nComputing item similarity matrix...")
    start_time = time.time()
    item_sim = compute_similarity_matrix(train_matrix.T, method='constrained_pearson', shrinkage=10)
    print(f"Item similarity matrix computed in {time.time() - start_time:.2f} seconds")

    print("Evaluating item-based CF...")
    start_time = time.time()
    y_true_i = []  # Create a new ground truth array specifically for item-based CF
    y_pred_i = []

    try:
        for _, row in tqdm(eval_test.iterrows(), total=len(eval_test)):
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_item(u, i, train_matrix, item_sim, k=20)
                if not np.isnan(pred):
                    y_true_i.append(row['rating'])  # Add the true rating
                    y_pred_i.append(pred)  # Add the prediction
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue
    except NameError:
        # If tqdm is not available
        for _, row in eval_test.iterrows():
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_item(u, i, train_matrix, item_sim, k=20)
                if not np.isnan(pred):
                    y_true_i.append(row['rating'])  # Add the true rating
                    y_pred_i.append(pred)  # Add the prediction
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue

    print(f"Item-based CF evaluation completed in {time.time() - start_time:.2f} seconds")
    if len(y_pred_i) > 0:
        print(f"Item-based CF RMSE: {rmse(np.array(y_pred_i), np.array(y_true_i)):.4f} (based on {len(y_pred_i)} predictions)")
    else:
        print("Item-based CF: No valid predictions were made.")

    # Matrix Factorization
    print("\nTraining matrix factorization model...")
    start_time = time.time()
    P, Q = matrix_factorization(train_matrix, k=20, alpha=0.005, beta=0.02, iterations=50)
    print(f"Matrix factorization training completed in {time.time() - start_time:.2f} seconds")

    print("Evaluating matrix factorization...")
    start_time = time.time()
    y_true_mf = []  # Create a new ground truth array specifically for matrix factorization
    y_pred_mf = []

    try:
        for _, row in tqdm(eval_test.iterrows(), total=len(eval_test)):
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_mf(u, i, train_matrix, P, Q)
                if not np.isnan(pred):
                    y_true_mf.append(row['rating'])  # Add the true rating
                    y_pred_mf.append(pred)  # Add the prediction
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue
    except NameError:
        for _, row in eval_test.iterrows():
            try:
                u = int(row['userId'])
                i = int(row['movieId'])
                pred = predict_rating_mf(u, i, train_matrix, P, Q)
                if not np.isnan(pred):
                    y_true_mf.append(row['rating'])  # Add the true rating
                    y_pred_mf.append(pred)  # Add the prediction
            except Exception as e:
                print(f"Error predicting for user {u}, item {i}: {e}")
                continue

    print(f"Matrix factorization evaluation completed in {time.time() - start_time:.2f} seconds")
    if len(y_pred_mf) > 0:
        print(f"Matrix Factorization RMSE: {rmse(np.array(y_pred_mf), np.array(y_true_mf)):.4f} (based on {len(y_pred_mf)} predictions)")
    else:
        print("Matrix Factorization: No valid predictions were made.")

    # Summary
    print("\nSummary of Results:")
    if len(y_pred_u) > 0:
        print(f"User-based CF RMSE: {rmse(np.array(y_pred_u), np.array(y_true)):.4f} (based on {len(y_pred_u)} predictions)")
    else:
        print("User-based CF: No valid predictions were made.")

    if len(y_pred_i) > 0:
        print(f"Item-based CF RMSE: {rmse(np.array(y_pred_i), np.array(y_true_i)):.4f} (based on {len(y_pred_i)} predictions)")
    else:
        print("Item-based CF: No valid predictions were made.")

    if len(y_pred_mf) > 0:
        print(f"Matrix Factorization RMSE: {rmse(np.array(y_pred_mf), np.array(y_true_mf)):.4f} (based on {len(y_pred_mf)} predictions)")
    else:
        print("Matrix Factorization: No valid predictions were made.")


if __name__ == "__main__":
    main()


Data loaded in 0.07 seconds
Taking a smaller sample for faster execution...
Splitting data into train/test sets...
Data split in 0.28 seconds
Train set: 29641 ratings, Test set: 610 ratings
Creating rating matrix...
Rating matrix created in 0.05 seconds
Matrix shape: (610, 6096)

Computing user similarity matrix...
Using a sample of 200 entities for similarity computation
User similarity matrix computed in 0.64 seconds
Evaluating user-based CF...


100%|██████████| 610/610 [00:00<00:00, 795.16it/s]


User-based CF evaluation completed in 0.80 seconds
User-based CF RMSE: 1.0571

Computing item similarity matrix...
Using a sample of 200 entities for similarity computation
Item similarity matrix computed in 0.19 seconds
Evaluating item-based CF...


100%|██████████| 610/610 [00:00<00:00, 2776.20it/s]


Item-based CF evaluation completed in 0.22 seconds
Item-based CF RMSE: 1.0826 (based on 610 predictions)

Training matrix factorization model...
Matrix factorization with 50 iterations...
  Iteration 1/50, RMSE: 3.6458
  Iteration 5/50, RMSE: 3.5530
  Iteration 10/50, RMSE: 1.9071
  Iteration 15/50, RMSE: 1.2791
  Iteration 20/50, RMSE: 1.0027
  Iteration 25/50, RMSE: 0.8496
  Iteration 30/50, RMSE: 0.7522
  Iteration 35/50, RMSE: 0.6823
  Iteration 40/50, RMSE: 0.6269
  Iteration 45/50, RMSE: 0.5809
  Iteration 50/50, RMSE: 0.5411
Matrix factorization training completed in 21.85 seconds
Evaluating matrix factorization...


100%|██████████| 610/610 [00:01<00:00, 416.21it/s]


Matrix factorization evaluation completed in 1.47 seconds
Matrix Factorization RMSE: 1.1485 (based on 610 predictions)

Summary of Results:
User-based CF RMSE: 1.0571 (based on 610 predictions)
Item-based CF RMSE: 1.0826 (based on 610 predictions)
Matrix Factorization RMSE: 1.1485 (based on 610 predictions)
